In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import cv2

In [2]:
from tensorflow.keras.utils import to_categorical

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#Numpy array conversion in Yoga_pose
X_train=np.load('/content/drive/MyDrive/yoga/X_train.npy')
X_test=np.load('/content/drive/MyDrive/yoga/X_test.npy')
y_train=np.load('/content/drive/MyDrive/yoga/y_train.npy')
y_test=np.load('/content/drive/MyDrive/yoga/y_test.npy')
y_to_check=np.load('/content/drive/MyDrive/yoga/y_to_check.npy')

In [5]:
#Normailizing the numpy array, both methods are fine.

# X_train=tf.keras.applications.resnet50.preprocess_input(X_train)
# X_test=tf.keras.applications.resnet50.preprocess_input(X_test)
X_train=X_train/255
X_test=X_test/255

In [6]:
# resize_and_rescale = tf.keras.Sequential([
#   layers.experimental.preprocessing.Resizing(224,224),
#   layers.experimental.preprocessing.Rescaling(1./255),
# ])

In [7]:
# Data Augmentation to counter Overfitting
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.RandomZoom(0.2),
  layers.RandomContrast(0.3)
])

In [8]:
#ResNet 152 for transfer Learning
base_model = tf.keras.applications.ResNet152(weights = 'imagenet',include_top = False, input_shape = (224,224,3))

#Unfreeze the layers so the model can learn weights according to the dataset.
# for layer in base_model.layers:
#   layer.trainable = False

In [9]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense1= layers.Dense(1000, activation='relu')
drop1= layers.Dropout(0.4)
dense2= layers.Dense(1000, activation='relu')
drop2= layers.Dropout(0.4)
prediction_layer = layers.Dense(37, activation='softmax')


base_model = models.Sequential([
    data_augmentation,
    base_model,
    flatten_layer,
    dense1,
    drop1,
    dense2,
    drop2,
    prediction_layer
])

In [10]:
loss = tf.keras.losses.CategoricalCrossentropy()
optim = tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics = ["accuracy"]

base_model.compile(optimizer=optim, loss=loss, metrics=metrics)

In [11]:
from tensorflow.keras.callbacks import EarlyStopping

In [12]:
#Defining a function to stop training when val_accuracy raches a certain threshold
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None): 
        accuracy = logs["val_accuracy"]
        if accuracy >= self.threshold:
            self.model.stop_training = True

In [13]:
callback=MyThresholdCallback(threshold=0.74) #train until val_accuracy is not 0.74

In [15]:
#For earlystopping we need to validation dataset
base_model.fit(X_train, y_train, epochs=150, validation_split=0.20, verbose=2, callbacks=[callback], shuffle = True)

Epoch 1/150
38/38 - 44s - loss: 5.1311 - accuracy: 0.0554 - val_loss: 3.6955 - val_accuracy: 0.0264 - 44s/epoch - 1s/step
Epoch 2/150
38/38 - 17s - loss: 3.4648 - accuracy: 0.1017 - val_loss: 3.6100 - val_accuracy: 0.0264 - 17s/epoch - 460ms/step
Epoch 3/150
38/38 - 17s - loss: 3.1648 - accuracy: 0.1671 - val_loss: 3.6023 - val_accuracy: 0.0231 - 17s/epoch - 460ms/step
Epoch 4/150
38/38 - 17s - loss: 2.8623 - accuracy: 0.2250 - val_loss: 3.7419 - val_accuracy: 0.0363 - 17s/epoch - 460ms/step
Epoch 5/150
38/38 - 18s - loss: 2.6045 - accuracy: 0.3077 - val_loss: 4.1127 - val_accuracy: 0.0198 - 18s/epoch - 461ms/step
Epoch 6/150
38/38 - 17s - loss: 2.2820 - accuracy: 0.3763 - val_loss: 3.8668 - val_accuracy: 0.0132 - 17s/epoch - 460ms/step
Epoch 7/150
38/38 - 17s - loss: 2.1128 - accuracy: 0.4103 - val_loss: 4.0856 - val_accuracy: 0.0363 - 17s/epoch - 460ms/step
Epoch 8/150
38/38 - 17s - loss: 1.7469 - accuracy: 0.5004 - val_loss: 4.0486 - val_accuracy: 0.0363 - 17s/epoch - 460ms/step
Epo

In [16]:
base_model.evaluate(X_test,y_test)

21/21 [==============================] - 3s 136ms/step - loss: 1.6139 - accuracy: 0.7257


[1.6139328479766846, 0.7257319092750549]

In [17]:
predictions = base_model.predict(X_test)

In [18]:
y_predicted = np.argmax(predictions, axis = 1)

In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_to_check, y_predicted))

              precision    recall  f1-score   support

           0       1.00      0.60      0.75         5
           1       0.69      0.75      0.72        12
           2       0.78      0.70      0.74        20
           3       0.75      0.75      0.75         4
           4       0.69      0.82      0.75        22
           5       0.86      1.00      0.92         6
           6       0.80      0.52      0.63        23
           7       0.80      0.84      0.82        19
           8       0.00      0.00      0.00         4
           9       1.00      0.20      0.33         5
          10       0.83      0.79      0.81        19
          11       0.52      0.75      0.62        16
          12       0.87      0.68      0.76        19
          13       0.82      0.64      0.72        22
          14       0.67      0.57      0.62        21
          15       1.00      0.50      0.67         4
          16       0.77      0.93      0.84        29
          17       0.94    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
